## Zero Inflated Poisson likelihood function:
##### offsetx and offsetz are hard coded as zero vectors; weights is hardcoded as a vector of 1s.
##### Only defined for logit link in binomial.

In [8]:
import numpy as np
import pandas as pd
import patsy
df = pd.read_csv('DebTrivedi.csv',index_col = [0])
sel = np.array([1, 6, 7, 8, 13, 15, 18])-1
df = df.iloc[:,sel]
# produce design matrices from R-style formula
formula = 'ofp ~ hosp + health + numchron + gender + school + privins'
y, X = patsy.dmatrices(formula, df, return_type='dataframe')

In [5]:
X.head()

,Intercept,health[T.excellent],health[T.poor],gender[T.male],privins[T.yes],hosp,numchron,school
1,1.0,0.0,0.0,1.0,1.0,1.0,2.0,6.0
2,1.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0
3,1.0,0.0,1.0,0.0,0.0,3.0,4.0,10.0
4,1.0,0.0,1.0,1.0,1.0,1.0,2.0,3.0
5,1.0,0.0,0.0,0.0,1.0,0.0,2.0,6.0


In [23]:
# health as Z variables
Z = X.iloc[:,[1,2]]

In [303]:
Y.shape

(4406, 1)

In [307]:
Y = np.squeeze(y.values)
## convenience variables
n = len(Y)
kx = X.shape[1] # Number of columns in X
kz = Z.shape[1]
Y0 = Y <= 0
Y1 = Y > 0

offsetx = np.zeros(n)
offsetz = np.zeros(n)
weights = np.ones(n)

In [308]:
type(weights)

numpy.ndarray

In [309]:
weights.shape

(4406,)

In [199]:
np.ones(10)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

## R functions:
    model_count <- glm.fit(X, Y, family = poisson(), weights = weights, offset = offsetx)
    model_zero <- glm.fit(Z, as.integer(Y0), weights = weights, family = binomial(link = linkstr), offset = offsetz)
    start <- list(count = model_count$coefficients, zero = model_zero$coefficients)

    ## set up likelihood
    ziPoisson <- function(parms) {
    ## count mean
    mu <- as.vector(exp(X %*% parms[1:kx] + offsetx))
    ## binary mean
    phi <- as.vector(linkinv(Z %*% parms[(kx+1):(kx+kz)] + offsetz))
    
    ## log-likelihood for y = 0 and y >= 1
    loglik0 <- log( phi + exp( log(1-phi) - mu ) ) ## -mu = dpois(0, lambda = mu, log = TRUE)
    loglik1 <- log(1-phi) + dpois(Y, lambda = mu, log = TRUE)
    
    ## collect and return
    loglik <- sum(weights[Y0] * loglik0[Y0]) + sum(weights[Y1] * loglik1[Y1])
    loglik
    }

In [42]:
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [310]:
model_count = sm.Poisson(endog = Y, exog = X).fit()

Optimization terminated successfully.
         Current function value: 4.078895
         Iterations 23


In [311]:
count = model_count.params

In [59]:
#model_zero = sm.Logit(endog = Y0, exog = Z).fit()
#zero = model_zero.params

Optimization terminated successfully.
         Current function value: 0.637490
         Iterations 6


In [312]:
model_zero = sm.GLM(Y0, exog = Z, family=sm.families.Binomial()).fit()
zero = model_zero.params

In [313]:
model_zero.predict()

array([ 0.5       ,  0.5       ,  0.10649819, ...,  0.5       ,
        0.5       ,  0.23906706])

In [ ]:
coef = {'zero':zero, 'count':count}

In [106]:
coef['zero']

health[T.excellent]   -1.157801
health[T.poor]        -2.127020
dtype: float64

In [314]:
aloha= np.hstack((count.values,zero.values))

In [315]:
aloha

array([ 1.0288742 , -0.3619932 ,  0.24830697, -0.11231992,  0.20168688,
        0.16479739,  0.14663928,  0.02614299, -1.15780116, -2.12702032])

In [148]:
parms = aloha

In [176]:
import scipy as sp
sp.special.expit(Z);

In [159]:
X.shape

(4406, 8)

In [164]:
kx-1

7

In [316]:
parms[[np.arange(kx)]]

array([ 1.0288742 , -0.3619932 ,  0.24830697, -0.11231992,  0.20168688,
        0.16479739,  0.14663928,  0.02614299])

In [171]:
np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)

array([  5.65859172,   5.9611865 ,  13.72968619, ...,  10.27743623,
         4.68458323,   1.74117657])

In [173]:
import scipy as sp
linkinv = sp.special.expit

#### ToDo: Define other link inverse functions or find relevant python libraries

In [177]:
linkinv(Z);

In [180]:
linkinv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)

array([ 0.5       ,  0.5       ,  0.10649819, ...,  0.5       ,
        0.5       ,  0.23906706])

In [333]:
def ziPoisson(parms):
    ## count mean
    mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
    ## binary mean
    phi = linkinv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
    ## log-likelihood for y = 0 and y >= 1
    loglik0 = np.log( phi + np.exp( np.log(1-phi) - mu ) ) ## -mu = dpois(0, lambda = mu, log = TRUE)
    loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)
    ## collect and return
    loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])
    return loglik

In [299]:
mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
## binary mean
phi = linkinv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
## log-likelihood for y = 0 and y >= 1
loglik0 = np.log( phi + np.exp( np.log(1-phi) - mu ) ) ## -mu = dpois(0, lambda = mu, log = TRUE)
loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)
loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])

In [302]:
np.dot(weights[Y0],loglik0[Y0])

-601.08719389080204

In [317]:
mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)

In [294]:
(np.dot(X,parms[np.arange(kx)])+offsetx).shape

(4406,)

In [290]:
mu = np.expand_dims(mu,axis=1)

In [318]:
mu.shape

(4406,)

In [319]:
phi = linkinv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)

In [320]:
phi.shape

(4406,)

In [325]:
loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)

In [326]:
loglik1.shape

(4406,)

In [322]:
loglik0 = np.log( phi + np.exp( np.log(1-phi) - mu ) )

In [323]:
loglik0.shape

(4406,)

In [328]:
loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])

In [330]:
np.dot(weights[Y0],loglik0[Y0])

-601.08719389080204

In [331]:
np.dot(weights[Y1],loglik1[Y1])

-16938.842337728696

In [332]:
loglik

-17539.929531619498

In [296]:
weights[Y0].shape

IndexError: too many indices for array

In [278]:
loglik0[Y0].shape

(683,)

In [264]:
mu.shape

(4406,)

In [279]:
loglik1[Y1].shape

(3723,)

In [265]:
phi.shape

(4406,)

In [267]:
Y.shape

(4406, 1)

In [266]:
sp.stats.poisson.logpmf(Y, mu).shape

(4406, 4406)

In [282]:
np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])

-17539.929531619498

In [281]:
np.dot(weights[Y1],loglik1[Y1])

-16938.842337728696

In [236]:
loglik0 = np.log( phi + np.exp( np.log(1-phi) - mu ) ) ## -mu = dpois(0, lambda = mu, log = TRUE)
loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)

In [244]:
mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)

In [245]:
mu.shape

(4406, 4406)

In [250]:
(np.dot(X,parms[np.arange(kx)])+ offsetx).shape

(4406, 4406)

In [237]:
np.sum(weights[Y0] * loglik0[Y0]) + np.sum(weights[Y1] * loglik1[Y1])

IndexError: too many indices for array

In [238]:
weights[Y0] * loglik0[Y0]

IndexError: too many indices for array

In [213]:
Y0.shape

(4406, 1)

In [231]:
weights[Y0].shape

(683,)

In [233]:
3723+683

4406

In [212]:
weights[Y0]

IndexError: too many indices for array

In [185]:
phi = linkinv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)

In [186]:
mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)

In [203]:
dpois = sp.stats.poisson(mu)

In [215]:
np.log(1-phi) + dpois.logpmf(Y)

array([[ -2.4733554 ,  -2.51547773,  -5.53198304, ...,  -4.10832098,
         -2.44383739,  -4.02907298],
       [ -4.61856385,  -4.86906414, -11.22273275, ...,  -8.64063258,
         -3.83345346,  -1.45982553],
       [ -6.37262713,  -5.99799354,  -2.34017224, ...,  -3.23338641,
         -7.85429387, -17.3572565 ],
       ..., 
       [ -4.12440099,  -3.90605087,  -2.75110205, ...,  -2.77548763,
         -5.03937345, -11.57318845],
       [ -9.29279824,  -8.76188117,  -2.6011874 , ...,  -4.36323016,
        -11.34115926, -23.81326954],
       [ -6.3517389 ,  -6.65433368, -13.84229312, ..., -10.97058341,
         -5.37773041,  -2.01438661]])

In [216]:
np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)

array([[ -2.4733554 ,  -2.51547773,  -5.53198304, ...,  -4.10832098,
         -2.44383739,  -4.02907298],
       [ -4.61856385,  -4.86906414, -11.22273275, ...,  -8.64063258,
         -3.83345346,  -1.45982553],
       [ -6.37262713,  -5.99799354,  -2.34017224, ...,  -3.23338641,
         -7.85429387, -17.3572565 ],
       ..., 
       [ -4.12440099,  -3.90605087,  -2.75110205, ...,  -2.77548763,
         -5.03937345, -11.57318845],
       [ -9.29279824,  -8.76188117,  -2.6011874 , ...,  -4.36323016,
        -11.34115926, -23.81326954],
       [ -6.3517389 ,  -6.65433368, -13.84229312, ..., -10.97058341,
         -5.37773041,  -2.01438661]])

In [179]:
np.arange(kx,kx+kz)

array([8, 9])

In [39]:
rep(0,10)

Couldn't evaluate or find in history: (0,10)


In [41]:
np.zeros(10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

## A note from stackoverflow:
The bad scaling deteriorates numerical algorithms even more in the case of GLM and other non-linear models because the nonlinearity can amplify the scaling.

statsmodels does not perform any automatic rescaling of the design matrix provided by the user. This means that in ill-conditioned cases we can get exceptions for singular matrix, results that are mostly numerical noise or convergence failures depending on the model that is used.

(Currently, only the linear models, OLS and similar, add a warning to the summary about small eigenvalues or large condition number. Overall, there is not enough checking of the design matrix to alert users about a "bad" design. That's still the responsibility of the user to check.)